In [1]:
# Spark Context 설정
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("restaurant-reviw-average")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/02 04:02:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# csv 파일을 RDD 로딩

In [5]:
filepath = "home/ubuntu/working/spark-examples/data/restaurant_reviews.csv"

In [6]:
lines = sc.textFile(f"file:///{filepath}")
lines

file:///home/ubuntu/working/spark-examples/data/restaurant_reviews.csv MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [7]:
# collect() : RDD 내의 데이터를 모두 출력
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [11]:
# header 제거
header = lines.first()
datas = lines.filter(lambda row : row != header)
datas

PythonRDD[7] at RDD at PythonRDD.scala:53

In [12]:
datas.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [13]:
def parse_tesk(row):
    # 2번, 3번 필드 가져오기 단 3번 필드는 정수로 리턴
    fields = row.split(",")
    category = fields[2]
    review_cnt = int(fields[3])
    
    return category, review_cnt

In [14]:
parse_tesk('0,짜장면,중식,125,')

('중식', 125)

In [16]:
# RDD 내의 모든 데이터에 대해 'parse_tesk' 함수를 적용후 추출(map)
category_reviews = datas.map(parse_tesk)
category_reviews

PythonRDD[8] at RDD at PythonRDD.scala:53

In [17]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [18]:
# 카테고리 별 리뷰 개수 평균
category_reviews_count = category_reviews.mapValues(lambda x: (x, 1))
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [20]:
## reduceBykey를 이용해 Key별 value에 대한 집계를 수행.
# 집계 Task 정의가 필요하다.(함수)

reduced = category_reviews_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [21]:
average =  reduced.mapValues(lambda x : x[0] / x[1])
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [22]:
sc.stop()